In [1]:
import sys,os
sys.path.append('../')
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.tools.coverage import Uncovered,UncoveredGroup
setLogLevel("error")
%load_ext line_profiler
# Set the path to the database
database = Database(os.path.expanduser("../test/database"))

### Load model

In [2]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 10*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

In [4]:
print(len(topDict.getElements()))
totWeight = None
for el in topDict.getElements():
    if totWeight is None:
        totWeight = el.weightList.copy()
    else:
        totWeight += el.weightList
print(totWeight)

170
['1.30E+01 [TeV]:8.40E+00 [pb] (None, None)', '8.00E+00 [TeV]:3.74E+00 [pb] (None, None)']


### Load the experimental results to be used.

In [5]:
listOfExpRes = database.getExpResults()

### Compute the theory predictions

In [6]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = []
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, topDict, combinedResults=False, marginalize=False)
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % expResult.globalInfo.id)
    for theoryPrediction in predictions:
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)


 Theory Predictions and Constraints:

 ATLAS-CONF-2013-037 
r = 1.093E-01

 ATLAS-SUSY-2013-02 
r = 6.581E-05


In [7]:
import time
t0 = time.time()
uncovered = Uncovered(topDict, sqrts=13.*TeV, sigmacut=20.0*fb)
print(time.time()-t0)

200.81980919837952


In [8]:
# First sort coverage groups by label
groups = sorted(uncovered.groups[:], key=lambda g: g.label)
# Print uncovered cross-sections:
for group in groups:
    print("\nTotal cross-section for %s (fb): %10.3E\n" % (group.description, group.getTotalXSec()))


Total cross-section for missing topologies (fb):  4.279E+03


Total cross-section for missing topologies with displaced decays (fb):  0.000E+00


Total cross-section for missing topologies with prompt decays (fb):  4.279E+03


Total cross-section for topologies outside the grid (fb):  4.719E+01



In [9]:
missingTopos = uncovered.getGroup('missing (prompt)')
# Print some of the missing topologies:
if missingTopos.finalStateElements:
    print('Missing topologies (up to 3):')
    for genEl in missingTopos.finalStateElements:
        print('Element:', genEl)
        print('\tcross-section (fb):', genEl.missingX)
        print(genEl == missingTopos.finalStateElements[0])
else:
    print("No missing topologies found\n")

Missing topologies (up to 3):
Element: (PV > MET,MET,jet,jet,l,nu)
	cross-section (fb): 617.7849796427008
True
Element: (PV > MET,MET,jet,jet,b,b)
	cross-section (fb): 507.53765000094455
False
Element: (PV > MET,MET,jet,jet)
	cross-section (fb): 349.99078730392665
False
Element: (PV > MET,MET,jet,jet,ta,nu)
	cross-section (fb): 308.0764717209802
False
Element: (PV > MET,MET,jet,jet,jet,jet,t,b,b,b)
	cross-section (fb): 245.50507478033944
False
Element: (PV > MET,MET,jet,jet,t,b,b,b)
	cross-section (fb): 240.28808395653778
False
Element: (PV > MET,MET,jet,jet,l,nu)
	cross-section (fb): 211.29193367176788
True
Element: (PV > MET,MET,jet,jet,b,b,b,b)
	cross-section (fb): 197.22754927958835
False
Element: (PV > MET,MET,jet,jet,jet,jet,b,b,b,b)
	cross-section (fb): 166.99846594764398
False
Element: (PV > MET,MET,jet,jet,l,nu)
	cross-section (fb): 165.54424053550974
True
Element: (PV > MET,MET,jet,jet,l,nu)
	cross-section (fb): 162.98943532740913
True
Element: (PV > MET,MET,jet,jet,ta,nu)
	c

In [10]:
missingDisplaced = uncovered.getGroup('missing (displaced)')
# Print elements with displaced decays:
if missingDisplaced.finalStateElements:
    print('\nElements with displaced vertices (up to 2):')
    for genEl in missingDisplaced.finalStateElements[:2]:
        print('Element:', genEl)
        print('\tcross-section (fb):', genEl.missingX)
else:
    print("\nNo displaced decays")



No displaced decays


In [11]:
for genEl in missingTopos.finalStateElements[:]:
    print('Element:', genEl)
    print('\tcross-section (fb):', genEl.missingX)
    for el in genEl._contributingElements:
        print(el.elID)
        print(el,el.weightList.getMaxXsec())


Element: (PV > MET,MET,jet,jet,l,nu)
	cross-section (fb): 617.7849796427008
16
(PV > C1-(1),C1+(2)), (C1-(1) > N1~,mu-,nu), (C1+(2) > N1,q,q) 7.72E-02 [pb]
17
(PV > C1-(1),C1+(2)), (C1-(1) > N1~,mu-,nu), (C1+(2) > N1,q,c) 7.72E-02 [pb]
24
(PV > C1-(1),C1+(2)), (C1-(1) > N1~,e-,nu), (C1+(2) > N1,q,q) 7.72E-02 [pb]
27
(PV > C1-(1),C1+(2)), (C1-(1) > N1~,e-,nu), (C1+(2) > N1,q,c) 7.72E-02 [pb]
30
(PV > C1+(1),C1-(2)), (C1+(1) > N1,nu,mu+), (C1-(2) > N1~,q,q) 7.72E-02 [pb]
32
(PV > C1+(1),C1-(2)), (C1+(1) > N1,nu,mu+), (C1-(2) > N1~,q,c) 7.72E-02 [pb]
33
(PV > C1+(1),C1-(2)), (C1+(1) > N1,nu,e+), (C1-(2) > N1~,q,q) 7.72E-02 [pb]
34
(PV > C1+(1),C1-(2)), (C1+(1) > N1,nu,e+), (C1-(2) > N1~,q,c) 7.72E-02 [pb]
Element: (PV > MET,MET,jet,jet,b,b)
	cross-section (fb): 507.53765000094455
75
(PV > C1+(1),N2(2)), (C1+(1) > N1,q,c), (N2(2) > N1,b,b) 1.62E-01 [pb]
76
(PV > C1+(1),N2(2)), (C1+(1) > N1,q,q), (N2(2) > N1,b,b) 1.62E-01 [pb]
43
(PV > C1-(1),N2(2)), (C1-(1) > N1~,q,c), (N2(2) > N1,b,b) 9.1

In [12]:
elList = topDict[111010100110101000]
for iel,el in enumerate(elList):
    if 'e+' not in str(el):
        continue
    if 'nu' not in str(el):
        continue
    if 'q' not in str(el):
        continue
    if 'N2' not in str(el):
        continue
    print(iel,el.elID,missingTopos.reweightFactor(el),missingTopos.elementFilter(el))
#     print(el)
    print(el,el.weightList.getMaxXsec())
    print('\t',el.testedBy,el.coveredBy)
    for eel in el.motherElements:
        print('\t',eel.elID)

69 70 1.0 True
(PV > C1+(1),N2(2)), (C1+(1) > N1,nu,e+), (N2(2) > N1,q,q) 4.07E-02 [pb]
	 set() set()
	 70
80 81 1.0 True
(PV > C1+(1),N2(2)), (C1+(1) > N1,nu,e+), (N2(2) > N1,q,q) 5.28E-02 [pb]
	 set() set()
	 81
85 86 1.0 True
(PV > C1+(1),N2(2)), (C1+(1) > N1,nu,e+), (N2(2) > N1,q,q) 5.28E-02 [pb]
	 set() set()
	 86
